In [1]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re

In [2]:
query = """SELECT mapbox_account_id, vertical
FROM (SELECT *, max(dt) over (partition by 'dummy') as max_dt from analytics.mbx_account_verticals)
where dt = max_dt
"""

In [3]:
formatter = column_parser.Parser(query)
formatted_query = formatter.format_query(query)
cte_queries = formatter.parse_cte(formatted_query)
cte_queries

{'main': "SELECT mapbox_account_id,\n       vertical\nFROM\n  (SELECT *,\n          max(dt) OVER (PARTITION BY 'dummy') AS max_dt\n   FROM analytics.mbx_account_verticals)\nWHERE dt = max_dt"}

In [4]:
db_fields = pd.DataFrame({'db_table': 'analytics.mbx_account_verticals', 
            'all_columns': ['mapbox_account_id', 'email', 'salesforce_account_id', 'vertical', 'vertical_source', 'dt']})


In [5]:
cte_queries

{'main': "SELECT mapbox_account_id,\n       vertical\nFROM\n  (SELECT *,\n          max(dt) OVER (PARTITION BY 'dummy') AS max_dt\n   FROM analytics.mbx_account_verticals)\nWHERE dt = max_dt"}

In [6]:
query = cte_queries['main']
columns_queried = formatter.match_queried_fields(query, db_fields)
columns_queried


[{'database_name': 'analytics',
  'table_name': 'mbx_account_verticals',
  'column_name': 'email'},
 {'database_name': 'analytics',
  'table_name': 'mbx_account_verticals',
  'column_name': 'mapbox_account_id'},
 {'database_name': 'analytics',
  'table_name': 'mbx_account_verticals',
  'column_name': 'salesforce_account_id'},
 {'database_name': 'analytics',
  'table_name': 'mbx_account_verticals',
  'column_name': 'vertical_source'},
 {'database_name': 'analytics',
  'table_name': 'mbx_account_verticals',
  'column_name': 'dt'},
 {'database_name': 'analytics',
  'table_name': 'mbx_account_verticals',
  'column_name': 'vertical'}]

In [ ]:
queried_cols

In [ ]:
# _map_db_columns
original_columns_list = []
var_list = variables
queried_cols = queried_columns

for var in set(var_list):

    var = var.strip(' ')

    if var in table_alias_mapping.keys():
        pass

    else:
        var_split = var.split('.')

        if len(var_split) == 1:
            if var_split[0] == '*':
                for db_table in queried_cols:
                    for k,v in db_table.items():
                        for col in v:
                            original_columns_list.append("{}.{}".format(k, col))
                
            else:
                for db_table in queried_cols:
                    for k,v in db_table.items():
                        if var in v:
                            original_columns_list.append("{}.{}".format(k, var))

        elif len(var_split) == 2:

            if var_split[0] in table_alias_mapping.keys():
                db_table = table_alias_mapping[var_split[0]]
                
                for db_table_col in queried_cols:
                    for k,v in db_table_col.items():

                        if k == db_table and var_split[1] in v:
                            original_columns_list.append("{}.{}".format(k, var_split[1]))
                        elif k == db_table and var_split[1] == '*':
                            for col in v:
                                original_columns_list.append("{}.{}".format(k, col))    

                                

In [ ]:
original_columns_list

In [ ]:
# _get_all_scanned_cols
all_columns_scanned = []

for _,cte_query in cte_queries.items():

    table_alias_mapping = formatter.get_table_names(cte_query.split('\n'))
    variables = formatter._get_all_variables(cte_query)
    queried_columns = formatter._get_queried_columns(table_alias_mapping, db_fields)
    
    if variables == []:
        original_columns_list = []
        for table in queried_columns:
            for k,v in table.items():
                for t in v:
                    original_columns_list.append("{}.{}".format(k,t))
    else:
        original_columns_list = formatter._map_db_columns(variables, queried_columns, table_alias_mapping)
    
    all_columns_scanned.extend(list(set(original_columns_list)))
    
    
    

In [ ]:
all_columns_scanned

In [ ]:
# match_queried_fields
all_columns_scanned = _get_all_scanned_cols(cte_queries, db_fields)

column_payload = []
for column in all_columns_scanned:
    try:
        col_split = column.split('.')
        db, table, col = col_split[0], col_split[1], col_split[2]
        row_payload = dict()
        row_payload["database_name"] = db
        row_payload["table_name"] = table 
        row_payload["column_name"] = col
        for arg, value in kwargs.items():
            row_payload[arg] = value    

        column_payload.append(row_payload)  
    except:
        pass